In [1]:
import tensorflow.compat.v1 as tf1
# import tensorflow as tf2
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
config = tf1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
session = tf1.Session(config=config)

2024-11-25 10:48:32.729355: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-25 10:48:32.774091: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-25 10:48:33.541174: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-11-25 10:48:34.752826: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20169 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:86:00.0, compute capab

In [2]:
import pandas as pd
import os
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
# .npy 파일 경로
x_train_path = 'x_train.npy'
y_train_path = 'y_train.npy'
x_val_path = 'x_val.npy'
y_val_path = 'y_val.npy'

# 데이터 불러오기
x_train = np.load(x_train_path)  # 훈련 이미지 데이터
y_train = np.load(y_train_path)  # 훈련 레이블 데이터
x_val = np.load(x_val_path)      # 검증 이미지 데이터
y_val = np.load(y_val_path)      # 검증 레이블 데이터

# 데이터 확인 (첫 번째 샘플 출력 예시)
print("훈련 이미지 데이터 크기:", x_train.shape)
print("훈련 레이블 데이터 크기:", y_train.shape)
print("검증 이미지 데이터 크기:", x_val.shape)
print("검증 레이블 데이터 크기:", y_val.shape)

훈련 이미지 데이터 크기: (737, 512, 512)
훈련 레이블 데이터 크기: (737, 512, 512, 2)
검증 이미지 데이터 크기: (185, 512, 512)
검증 레이블 데이터 크기: (185, 512, 512, 2)


In [4]:
def plot_accuracy_loss(history):
    """
    훈련 및 검증 정확도와 손실을 시각화합니다.
    
    매개변수:
    - history: 정확도 및 손실 값을 포함하는 훈련 기록 객체.
    """
    plt.figure(figsize=(10, 4))

    # 정확도 그래프
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Model Accuracy')

    # 손실 그래프
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Model Loss')

    plt.tight_layout()
    plt.show()

def calculate_f1_score(model, x_val, y_val, batch_size=4):
    """
    F1 점수를 계산하고 출력합니다.
    
    매개변수:
    - model: 검증 데이터 세트에서 예측을 생성할 훈련된 모델.
    - x_val: 검증 데이터 입력값.
    - y_val: 검증 데이터 레이블.
    - batch_size: 예측을 위한 배치 크기(기본값: 4).
    
    반환값:
    - y_pred: 예측된 레이블.
    - y_true: 실제 레이블.
    """
    y_pred = np.argmax(model.predict(x_val, batch_size=batch_size), axis=-1)
    y_true = np.argmax(y_val, axis=-1)

    f1 = f1_score(y_true.flatten(), y_pred.flatten(), average='weighted')
    print(f"F1 점수: {f1:.2f}")

    return y_true, y_pred

def plot_confusion_matrix_with_metrics(y_true, y_pred):
    """
    혼동 행렬을 시각화하고 정밀도, 재현율, F1 점수를 출력합니다.
    
    매개변수:
    - y_true: 실제 레이블.
    - y_pred: 예측된 레이블.
    """
    # Flatten 데이터
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    
    # 혼동 행렬 계산
    cm = confusion_matrix(y_true, y_pred)
    
    # 메트릭 계산 (zero_division=0 설정)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    
    # 메트릭 출력
    print("=== 성능 지표 ===")
    print(f"정밀도(Precision): {precision:.2f}")
    print(f"재현율(Recall): {recall:.2f}")
    
    # 혼동 행렬 시각화
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap='Blues', values_format='d')
    plt.title('Confusion Matrix')
    plt.show()
    
def plot_true_vs_pred_images(model, x_val, y_val, num_images=5, batch_size=4):
    """
    실제 이미지, 라벨 이미지 및 예측된 이미지를 나란히 시각화합니다.
    
    매개변수:
    - model: 검증 데이터 세트에서 예측을 생성할 훈련된 모델.
    - x_val: 검증 데이터 입력값.
    - y_val: 검증 데이터 레이블.
    - num_images: 시각화할 이미지 수 (기본값: 5).
    - batch_size: 예측을 위한 배치 크기(기본값: 4).
    """
    # 예측 생성
    y_pred = model.predict(x_val, batch_size=batch_size)
    
    plt.figure(figsize=(25, 3 * num_images))
    for i in range(num_images):
        # 실제 입력 이미지
        plt.subplot(num_images, 5, 5 * i + 1)
        plt.imshow(x_val[i].squeeze(), cmap='gray')
        plt.title('Original Image')
        plt.axis('off')
        
        # 실제 라벨 이미지 (첫 번째 채널 선택 - 전경)
        plt.subplot(num_images, 5, 5 * i + 2)
        plt.imshow(y_val[i][..., 1].squeeze(), cmap='gray')  # 전경 채널 시각화
        plt.title('Label Image (Foreground)')
        plt.axis('off')
        
        # 예측된 이미지 (전체 클래스 비교 - argmax로 클래스 라벨 선택)
        plt.subplot(num_images, 5, 5 * i + 3)
        plt.imshow(np.argmax(y_val[i], axis=-1).squeeze(), cmap='gray')
        plt.title('Original Label (Argmax)')
        plt.axis('off')
        
        # 예측된 이미지 (첫 번째 채널 선택 - 전경)
        plt.subplot(num_images, 5, 5 * i + 4)
        plt.imshow(y_pred[i][..., 1].squeeze(), cmap='gray')  # 전경 채널 시각화
        plt.title('Predicted Image (Foreground)')
        plt.axis('off')
        
        # 예측된 이미지 (전체 클래스 비교 - argmax로 클래스 라벨 선택)
        plt.subplot(num_images, 5, 5 * i + 5)
        plt.imshow(np.argmax(y_pred[i], axis=-1).squeeze(), cmap='gray')
        plt.title('Predicted Label (Argmax)')
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
def main_display_result(history, model, x_val, y_val, batch_size=4, num_images=5):
    """
    전체 워크플로우 실행: 정확도/손실 그래프 시각화, F1 점수 계산, 혼동 행렬 및 메트릭 출력, 실제/예측 이미지 비교.
    
    매개변수:
    - history: 정확도 및 손실 값을 포함하는 훈련 기록 객체.
    - model: 검증 데이터 세트에서 예측을 생성할 훈련된 모델.
    - x_val: 검증 데이터 입력값.
    - y_val: 검증 데이터 레이블.
    - batch_size: 예측을 위한 배치 크기(기본값: 4).
    - num_images: 시각화할 이미지 수 (기본값: 5).
    """
    # 훈련 및 검증 정확도/손실 그래프 시각화
    plot_accuracy_loss(history)
    
    # F1 점수 계산 및 예측값 반환
    y_true, y_pred = calculate_f1_score(model, x_val, y_val, batch_size=batch_size)
    
    # 혼동 행렬 및 메트릭 출력
    plot_confusion_matrix_with_metrics(y_true, y_pred)
    
    # 실제 이미지와 예측된 이미지 비교 시각화
    plot_true_vs_pred_images(model, x_val, y_val, num_images=num_images, batch_size=batch_size)

In [5]:
import random
from deap import base, creator, tools
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
import numpy as np

from tensorflow.keras.layers import Cropping2D

from tensorflow.keras.layers import ZeroPadding2D

def unet_model_dynamic(input_shape, num_classes, num_layers, filters_per_layer):
    # 입력 크기 정수화
    input_shape = tuple(map(int, input_shape))  # Ensure input_shape is integer
    inputs = Input(input_shape)

    # Encoder
    encoder_layers = []
    x = inputs
    for i in range(num_layers):
        x = Conv2D(filters_per_layer[i], (3, 3), activation='relu', padding='same')(x)
        x = Conv2D(filters_per_layer[i], (3, 3), activation='relu', padding='same')(x)
        encoder_layers.append(x)
        if i < num_layers - 1:
            x = MaxPooling2D((2, 2))(x)

    # Bottleneck
    bottleneck = Conv2D(filters_per_layer[-1], (3, 3), activation='relu', padding='same')(x)
    bottleneck = Conv2D(filters_per_layer[-1], (3, 3), activation='relu', padding='same')(bottleneck)

    # Decoder
    x = bottleneck
    for i in range(num_layers - 1, -1, -1):
        x = Conv2DTranspose(filters_per_layer[i], (2, 2), strides=(2, 2), padding='same')(x)

        # 크기 조정
        diff_height = encoder_layers[i].shape[1] - x.shape[1]
        diff_width = encoder_layers[i].shape[2] - x.shape[2]

        # 정수형 변환
        diff_height = int(diff_height)
        diff_width = int(diff_width)

        # ZeroPadding2D 또는 Cropping2D를 이용하여 크기 조정
        if diff_height > 0 or diff_width > 0:
            x = ZeroPadding2D(((diff_height // 2, diff_height - diff_height // 2),
                               (diff_width // 2, diff_width - diff_width // 2)))(x)
        elif diff_height < 0 or diff_width < 0:
            x = Cropping2D(((-diff_height // 2, -diff_height + diff_height // 2),
                            (-diff_width // 2, -diff_width + diff_width // 2)))(x)

        x = concatenate([x, encoder_layers[i]])
        x = Conv2D(filters_per_layer[i], (3, 3), activation='relu', padding='same')(x)
        x = Conv2D(filters_per_layer[i], (3, 3), activation='relu', padding='same')(x)

    # Output
    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(x)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

In [6]:
# 패치 단위 학습 함수 (필요시 수정 가능)
def train_unet_with_patches(model, x_train, y_train, x_val, y_val, patch_size, batch_size, epochs):
    return model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=batch_size, epochs=epochs)

In [7]:
# 적합도 평가 함수
def evaluate(individual):
    lr, batch_size, patch_size, num_layers, *filters_per_layer = individual

    # 필터 크기를 정수로 변환
    filters_per_layer = [int(f) for f in filters_per_layer]

    # 모델 생성
    model = unet_model_dynamic(
        input_shape=(patch_size, patch_size, 1),
        num_classes=2,
        num_layers=num_layers,
        filters_per_layer=filters_per_layer
    )
    optimizer = Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

    # 패치 단위 학습
    history = train_unet_with_patches(
        model,
        x_train, y_train, x_val, y_val,
        patch_size=(patch_size, patch_size),
        batch_size=batch_size,
        epochs=3
    )

    val_loss = history.history["val_loss"][-1]
    return val_loss,

# 유전 알고리즘 설정
param_ranges = {
    "learning_rate": (1e-5, 1e-2),
    "batch_size": (4, 16),
    "patch_size": (32, 128),
    "num_layers": (3, 5),
    "filters": (32, 128)
}

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_lr", random.uniform, *param_ranges["learning_rate"])
toolbox.register("attr_batch_size", random.randint, *param_ranges["batch_size"])
toolbox.register("attr_patch_size", random.randint, *param_ranges["patch_size"])
toolbox.register("attr_num_layers", random.randint, *param_ranges["num_layers"])
toolbox.register("attr_filters", random.randint, *param_ranges["filters"])

toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_lr, toolbox.attr_batch_size, toolbox.attr_patch_size,
                  toolbox.attr_num_layers) + (toolbox.attr_filters,) * param_ranges["num_layers"][1])
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)


In [8]:
# 실행
population = toolbox.population(n=10)
NGEN = 5
for gen in range(NGEN):
    print(f"Generation {gen}")
    offspring = tools.selBest(population, k=len(population)//2)
    offspring = list(map(toolbox.clone, offspring))

    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < 0.5:
            toolbox.mate(child1, child2)
            del child1.fitness.values, child2.fitness.values

    for mutant in offspring:
        if random.random() < 0.2:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    population[:] = offspring

    for ind in population:
        if not ind.fitness.valid:
            ind.fitness.values = toolbox.evaluate(ind)

    fits = [ind.fitness.values[0] for ind in population]
    print(f"Best fitness: {min(fits)}")

# 최적 개체 확인
best_individual = tools.selBest(population, k=1)[0]
print("Best Individual:", best_individual)

Generation 0


2024-11-25 10:48:37.668735: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20169 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:86:00.0, compute capability: 8.0


ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 8, 8, 103), (None, 7, 7, 103)]